# Data Cleaning 

#### 1. Import pandas library.

In [1]:
import pandas as pd

#### 2. Import the users table.

In [29]:
users = pd.read_csv("data/users.csv")

#### 3. Rename Id column to userId.

In [3]:
users.head() # the column already has the correct, hence I haven´t changed it. 

,Unnamed: 0,userId,Reputation,Views,UpVotes,DownVotes
0,0,-1,1,0,5007,1920
1,1,2,101,25,3,0
2,2,3,101,22,19,0
3,3,4,101,11,0,0
4,4,5,6792,1145,662,5


#### 4. Import the posts table. 

In [33]:
posts = pd.read_csv("data/posts.csv")

#### 5. Rename Id column to postId and OwnerUserId to userId.

In [12]:
posts.head()

,Unnamed: 0,PostId,userId,Score,ViewCount,CommentCount
0,0,1,8.0,23,1278.0,1
1,1,2,24.0,22,8198.0,1
2,2,3,18.0,54,3613.0,4
3,3,4,23.0,13,5224.0,2
4,4,5,23.0,81,NaN,3


In [34]:
posts_up = posts.rename(columns={"PostId":"postid"})

In [35]:
posts_up

,Unnamed: 0,postid,userId,Score,ViewCount,CommentCount
0,0,1,8.0,23,1278.0,1
1,1,2,24.0,22,8198.0,1
2,2,3,18.0,54,3613.0,4
3,3,4,23.0,13,5224.0,2
4,4,5,23.0,81,NaN,3
...,...,...,...,...,...,...
91971,91971,115374,805.0,2,NaN,2
91972,91972,115375,49365.0,0,9.0,0
91973,91973,115376,55746.0,1,5.0,2
91974,91974,115377,805.0,0,NaN,0


#### 6. Define new dataframes for users and posts with the following selected columns:
**users_sliced columns**: userId, Reputation, Views, UpVotes, DownVotes  
**posts_sliced columns**: postId, Score, userId, ViewCount, CommentCount

In [39]:
users_sliced_columns = users.drop(["Unnamed: 0"], axis=1) #create a new variable where I drop the column "Unnamed" from the df users

In [40]:
posts_sliced_columns = posts.drop(["Unnamed: 0"], axis=1) # same as above but with the df posts 

#### 7. Merge the two dataframes created in the step above (6), users_sliced and posts_sliced. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [2]:
combine_tables = pd.merge(posts_sliced_columns, users_sliced_columns, on="userId")  

NameError: name 'pd' is not defined

In [99]:
combine_tables

,PostId,userId,Score,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1,8.0,23,1278.0,1,6764,1089,604,25
1,16,8.0,16,NaN,3,6764,1089,604,25
2,36,8.0,41,67396.0,7,6764,1089,604,25
3,65,8.0,14,NaN,3,6764,1089,604,25
4,78,8.0,33,NaN,4,6764,1089,604,25
...,...,...,...,...,...,...,...,...,...
90579,115366,55742.0,1,17.0,0,6,0,0,0
90580,115370,55744.0,1,13.0,2,6,1,0,0
90581,115371,35801.0,0,19.0,0,1,1,0,0
90582,115375,49365.0,0,9.0,0,1,0,0,0


#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [103]:
combine_tables.isnull().sum()

PostId              0
userId              0
Score               0
ViewCount       48396
CommentCount        0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before going to the next step.

In [ ]:
# I would fill them with the average value of the ViewCount, this way the mean stays the same.

In [1]:
combine_tables["ViewCount"] = combine_tables["ViewCount"].mean()

NameError: name 'combine_tables' is not defined

In [133]:
average

556.6561581492367

In [129]:
update_table = combine_tables.fillna(average, axis=1)

In [130]:
update_table

,PostId,userId,Score,ViewCount,CommentCount,Reputation,Views,UpVotes,DownVotes
0,1,8,23,1278.000000,1,6764,1089,604,25
1,16,8,16,556.656158,3,6764,1089,604,25
2,36,8,41,67396.000000,7,6764,1089,604,25
3,65,8,14,556.656158,3,6764,1089,604,25
4,78,8,33,556.656158,4,6764,1089,604,25
...,...,...,...,...,...,...,...,...,...
90579,115366,55742,1,17.000000,0,6,0,0,0
90580,115370,55744,1,13.000000,2,6,1,0,0
90581,115371,35801,0,19.000000,0,1,1,0,0
90582,115375,49365,0,9.000000,0,1,0,0,0


In [107]:
update_table["ViewCount"].mean()

556.6561581492367

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [ ]:
# I would convert all the data to integers, so userId and ViewCount that are floats I´ll convert them to int. 

In [100]:
combine_tables.dtypes

PostId            int64
userId          float64
Score             int64
ViewCount       float64
CommentCount      int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
dtype: object

In [101]:
combine_tables["userId"] = combine_tables["userId"].astype(int)

In [134]:
combine_tables["ViewCount"] = combine_tables["ViewCount"].astype(int)

In [135]:
combine_tables.dtypes

PostId          int64
userId          int64
Score           int64
ViewCount       int64
CommentCount    int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
dtype: object